## Collecting protocol fees across Balancer core pools on all networks
- Spreadsheet as reference: https://docs.google.com/spreadsheets/d/1xwUPpbYq7woVOU9vQ8EB8MY75I-1mauTLyDVwvKUDKo/edit#gid=0
- Collab: https://colab.research.google.com/drive/1vKCvcV5mkL1zwW3565kLSGkBEbt8NsoB?usp=sharing


In [1]:
import datetime
import os

import pandas as pd
from dotenv import load_dotenv
from web3 import Web3
from web3.middleware import geth_poa_middleware

from notebooks import get_block_by_ts
from notebooks import get_twap_bpt_price
from notebooks.constants import ARB_CORE_POOLS
from notebooks.constants import BASE_CORE_POOLS
from notebooks.constants import GNOSIS_CORE_POOLS
from notebooks.constants import MAINNET_CORE_POOLS
from notebooks.constants import POLYGON_CORE_POOLS
from notebooks.constants import AVAX_CORE_POOLS

load_dotenv()
arb_web3 = Web3(Web3.HTTPProvider(os.environ["ARBNODEURL"]))
eth_web3 = Web3(Web3.HTTPProvider(os.environ["ETHNODEURL"]))
base_web3 = Web3(Web3.HTTPProvider(os.environ["BASENODEURL"]))
gnosis_web3 = Web3(Web3.HTTPProvider(os.environ["GNOSISNODEURL"]))
avax_web3 = Web3(Web3.HTTPProvider(os.environ["AVALANCHENODEURL"]))
poly_web3 = Web3(Web3.HTTPProvider("https://polygon-rpc.com"))
poly_web3.middleware_onion.inject(geth_poa_middleware, layer=0)

# ARBITRUM
# arb_block_now = arb_web3.eth.block_number - 1000
timestamp_now = 1694790000
arb_block = get_block_by_ts(timestamp_now, "arbitrum")  # 18 August 2023
# Given Arb block time, we want to look back 2 weeks:
arb_timestamp_2_weeks_ago = timestamp_now - (2 * 7 * 24 * 60 * 60)
arb_block_2_weeks_ago = get_block_by_ts(arb_timestamp_2_weeks_ago, "arbitrum")
# Convert to datetime:
arb_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
arb_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(arb_timestamp_2_weeks_ago)
# MAINNET
mainnet_block = get_block_by_ts(timestamp_now, "mainnet")
# Given mainnet block time, we want to look back 2 weeks:
mainnet_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
mainnet_block_2_weeks_ago = get_block_by_ts(mainnet_timestamp_2_weeks_ago, "mainnet")
# Convert to datetime:
mainnet_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
mainnet_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(mainnet_timestamp_2_weeks_ago)

# BASE
base_block = get_block_by_ts(timestamp_now, "base")
# Given base block time, we want to look back 2 weeks:
base_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
base_block_2_weeks_ago = get_block_by_ts(base_timestamp_2_weeks_ago, "base")
# Convert to datetime:
base_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
base_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(base_timestamp_2_weeks_ago)

# POLYGON
poly_block = get_block_by_ts(timestamp_now, "polygon")
# Given polygon block time, we want to look back 2 weeks:
poly_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
poly_block_2_weeks_ago = get_block_by_ts(poly_timestamp_2_weeks_ago, "polygon")
# Convert to datetime:
poly_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
poly_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(poly_timestamp_2_weeks_ago)

# GNOSIS
gnosis_block = get_block_by_ts(timestamp_now, "gnosis")
# Given gnosis block time, we want to look back 2 weeks:
gnosis_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
gnosis_block_2_weeks_ago = get_block_by_ts(gnosis_timestamp_2_weeks_ago, "gnosis")
# Convert to datetime:
gnosis_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
gnosis_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(gnosis_timestamp_2_weeks_ago)

# Avalanche
avax_block = get_block_by_ts(timestamp_now, "avalanche")
# Given avalanche block time, we want to look back 2 weeks:
avax_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
avax_block_2_weeks_ago = get_block_by_ts(avax_timestamp_2_weeks_ago, "avalanche")
# Convert to datetime:
avax_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
avax_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(avax_timestamp_2_weeks_ago)

print("Collecting bpt prices for Avalanche")
avax_bpt_twap_prices = {}
for avax_pool in AVAX_CORE_POOLS:
    avax_bpt_twap_prices[avax_pool] = get_twap_bpt_price(avax_pool, 'avalanche', avax_web3,
                                                         start_date=avax_datetime_now, block_number=avax_block)
print("Collecting bpt prices for Base")
base_bpt_twap_prices = {}
for base_pool in BASE_CORE_POOLS:
    base_bpt_twap_prices[base_pool] = get_twap_bpt_price(base_pool, 'base', base_web3,
                                                         start_date=base_datetime_now, block_number=base_block)

print("Collecting bpt prices for Gnosis")
gnosis_bpt_twap_prices = {}
for gnosis_pool in GNOSIS_CORE_POOLS:
    gnosis_bpt_twap_prices[gnosis_pool] = get_twap_bpt_price(gnosis_pool, 'gnosis', gnosis_web3,
                                                             start_date=gnosis_datetime_now,
                                                             block_number=gnosis_block)
print("Collecting bpt prices for Arbitrum")
arb_bpt_twap_prices = {}
for arb_pool in ARB_CORE_POOLS:
    arb_bpt_twap_prices[arb_pool] = get_twap_bpt_price(arb_pool, 'arbitrum', arb_web3,
                                                       start_date=arb_datetime_now,
                                                       block_number=arb_block)

print("Collecting bpt prices for Mainnet")
mainnet_bpt_twap_prices = {}
for mainnet_pool in MAINNET_CORE_POOLS:
    mainnet_bpt_twap_prices[mainnet_pool] = get_twap_bpt_price(mainnet_pool, 'mainnet', eth_web3,
                                                               start_date=mainnet_datetime_now,
                                                               block_number=mainnet_block)
print("Collecting bpt prices for Polygon")
polygon_bpt_twap_prices = {}
for polygon_pool in POLYGON_CORE_POOLS:
    polygon_bpt_twap_prices[polygon_pool] = get_twap_bpt_price(polygon_pool, 'polygon', poly_web3,
                                                               start_date=poly_datetime_now,
                                                               block_number=poly_block)

# Convert to dataframe and print, merge all three dataframes
arb_bpt_twap_prices_df = pd.DataFrame.from_dict(arb_bpt_twap_prices, orient='index')
mainnet_bpt_twap_prices_df = pd.DataFrame.from_dict(mainnet_bpt_twap_prices, orient='index')
polygon_bpt_twap_prices_df = pd.DataFrame.from_dict(polygon_bpt_twap_prices, orient='index')
base_bpt_twap_prices_df = pd.DataFrame.from_dict(base_bpt_twap_prices, orient='index')
gnosis_bpt_twap_prices_df = pd.DataFrame.from_dict(gnosis_bpt_twap_prices, orient='index')
avax_bpt_twap_prices_df = pd.DataFrame.from_dict(avax_bpt_twap_prices, orient='index')

In [2]:
# Arbitrum TWAP prices:
arb_bpt_twap_prices_df

,0
0xade4a71bb62bec25154cfc7e6ff49a513b491e81000000000000000000000497,1623.675371501289025353485139
0x9791d590788598535278552eecd4b211bfc790cb000000000000000000000498,1624.252035535099417229892070
0x4a2f6ae7f3e5d715689530873ec35593dc28951b000000000000000000000481,1628.204232474774112014425454
0x423a1323c871abc9d89eb06855bf5347048fc4a5000000000000000000000496,1.000287514715826949194563748
0x32df62dc3aed2cd6224193052ce665dc181658410002000000000000000003bd,1.084432736050647730628563298
0x0c8972437a38b389ec83d1e666b69b8a4fcf8bfd00000000000000000000049e,1622.940959239087596069245535


In [3]:
# Mainnet TWAP prices:
mainnet_bpt_twap_prices_df

,0
0x93d199263632a4ef4bb438f1feb99e57b4b5f0bd0000000000000000000005c2,1623.587005107450579662024221
0x1e19cf2d73a72ef1332c882f20534b6519be0276000200000000000000000112,1663.934974113912044024917831
0xe7e2c68d3b13d905bbb636709cf4dfd21076b9d20000000000000000000005ca,1608.222259958607013581801808
0xf7a826d47c8e02835d94fb0aa40f0cc9505cb1340002000000000000000005e0,1622.785695571185882560378691
0xf16aee6a71af1a9bc8f56975a4c2705ca7a782bc0002000000000000000004bb,24.91858221742486618071391902
0xb08885e6026bab4333a80024ec25a1a3e1ff2b8a000200000000000000000445,1636.606552338577880903796201
0x8353157092ed8be69a9df8f95af097bbf33cb2af0000000000000000000005d9,0.9829552482987328765622931103
0xdfe6e7e18f6cc65fa13c8d8966013d4fda74b6ba000000000000000000000558,1631.944956608841749485467565
0x5f1f4e50ba51d723f12385a8a9606afc3a0555f5000200000000000000000465,52.98363011851058701154733909
0x1ee442b5326009bb18f2f472d3e0061513d1a0ff000200000000000000000464,59.13604626968661896729490181


In [4]:
# Polygon TWAP prices:
polygon_bpt_twap_prices_df

,0
0xf0ad209e2e969eaaa8c882aac71f02d8a047d5c2000200000000000000000b49,0.5398874716669745596858350491
0xee278d943584dd8640eaf4cc6c7a5c80c0073e85000200000000000000000bc7,0.5377540163466937217803884392
0x89b753153678bc434c610b7e9182297cada8ff29000000000000000000000c21,0.5353766213457433438918130946
0xcd78a20c597e367a4e478a2411ceb790604d7c8f000000000000000000000c22,0.5354327861913115678237352158
0xdc31233e09f3bf5bfe5c10da2014677c23b6894c000000000000000000000c23,1622.743046126021349781091215


In [5]:
# Base TWAP prices:
base_bpt_twap_prices_df

,0
0xfb4c2e6e6e27b5b4a07a36360c89ede29bb3c9b6000000000000000000000026,1623.307948866703366372783373
0xc771c1a5905420daec317b154eb13e4198ba97d0000000000000000000000023,1625.183674834590786350316724
0x0c659734f1eef9c63b7ebdf78a164cdd745586db000000000000000000000046,1.000332568373425896848516494


In [6]:
# Gnosis TWAP prices:
gnosis_bpt_twap_prices_df

,0
0xbad20c15a773bf03ab973302f61fabcea5101f0a000000000000000000000034,1621.768685473807877459906674


In [7]:
# Avalanche TWAP prices:
avax_bpt_twap_prices_df

,0
0xfd2620c9cfcec7d152467633b3b0ca338d3d78cc00000000000000000000001c,9.687140483338847650524265929
0xc13546b97b9b1b15372368dc06529d7191081f5b00000000000000000000001d,9.728419493138605676084386486
0x9fa6ab3d78984a69e712730a2227f20bcc8b5ad900000000000000000000001f,9.691443562178658811206475307
0xb26f0e66317846bd5fe0cbaa1d269f0efeb05c9600000000000000000000001e,1.000015981715009858727387721
0x55bec22f8f6c69137ceaf284d9b441db1b9bfedc000200000000000000000011,0.9664020968316878016512963205


In [8]:
# Query:
POOLS_SNAPSHOTS_QUERY = """
{{
  poolSnapshots(
    first: {first}
    skip: {skip}
    orderBy: timestamp
    orderDirection: desc
    block: {{ number: {block} }}
    where: {{ protocolFee_not: null }}
  ) {{
    pool {{
      address
      id
      symbol
      totalProtocolFeePaidInBPT
      tokens {{
        symbol
        address
        paidProtocolFees
      }}
    }}
    timestamp
    protocolFee
    swapFees
    swapVolume
    liquidity
  }}
}}
"""

## Fetching data from the Balancer subgraphs

In [9]:
from notebooks.constants import BASE_BALANCER_GRAPH_URL
from notebooks.constants import POLYGON_BALANCER_GRAPH_URL
from notebooks.constants import ARB_BALANCER_GRAPH_URL
from notebooks.constants import MAINNET_BALANCER_GRAPH_URL
from notebooks.constants import GNOSIS_BALANCER_GRAPH_URL
from notebooks.constants import AVALANCHE_BALANCER_GRAPH_URL
from typing import Dict
from typing import List
from typing import Optional

from gql import Client
from gql import gql
from gql.transport.requests import RequestsHTTPTransport


# Fetch all the data from the balancer subgraph
def make_gql_client(url: str) -> Optional[Client]:
    transport = RequestsHTTPTransport(url=url, retries=3)
    return Client(
        transport=transport, fetch_schema_from_transport=True, execute_timeout=60
    )


def get_balancer_pool_snapshots(block: int, graph_url: str) -> Optional[List[Dict]]:
    client = make_gql_client(graph_url)
    all_pools = []
    limit = 1000
    offset = 0
    while True:
        result = client.execute(
            gql(POOLS_SNAPSHOTS_QUERY.format(first=limit, skip=offset, block=block)))
        all_pools.extend(result['poolSnapshots'])
        offset += limit
        if offset >= 5000:
            break
        if len(result['poolSnapshots']) < limit - 1:
            break
    return all_pools


arbi_pool_snapshots_now = get_balancer_pool_snapshots(arb_block, ARB_BALANCER_GRAPH_URL)
arbi_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(arb_block_2_weeks_ago), ARB_BALANCER_GRAPH_URL)

mainnet_pool_snapshots_now = get_balancer_pool_snapshots(mainnet_block, MAINNET_BALANCER_GRAPH_URL)
mainnet_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(mainnet_block_2_weeks_ago),
                                                                 MAINNET_BALANCER_GRAPH_URL)

polygon_pool_snapshots_now = get_balancer_pool_snapshots(poly_block, POLYGON_BALANCER_GRAPH_URL)
polygon_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(poly_block_2_weeks_ago),
                                                                 POLYGON_BALANCER_GRAPH_URL)

base_pool_snapshots_now = get_balancer_pool_snapshots(base_block, BASE_BALANCER_GRAPH_URL)
base_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(base_block_2_weeks_ago),
                                                              BASE_BALANCER_GRAPH_URL)

gnosis_pool_snapshots_now = get_balancer_pool_snapshots(gnosis_block, GNOSIS_BALANCER_GRAPH_URL)
gnosis_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(gnosis_block_2_weeks_ago),
                                                                GNOSIS_BALANCER_GRAPH_URL)

avax_pool_snapshots_now = get_balancer_pool_snapshots(avax_block, AVALANCHE_BALANCER_GRAPH_URL)
avax_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(avax_block_2_weeks_ago),
                                                              AVALANCHE_BALANCER_GRAPH_URL)

## Extract fee data for CORE pools:


In [10]:
from collections import defaultdict
from decimal import Decimal
from notebooks import fetch_token_price_balgql


def collect_fee_info(pools: list[str], chain: str, pools_now: list[dict], pools_shifted: list[dict],
                     start_date: datetime.datetime) -> tuple[dict, dict]:
    # Iterate through snapshots now and 2 weeks ago and extract fee data, by subtracting today's fee data from 2 weeks ago
    # and then summing across all pools
    fees = {}
    token_fees = defaultdict(list)
    for pool in pools:
        current_fees_snapshots = [x for x in pools_now if x['pool']['id'] == pool]
        current_fees_snapshots.sort(key=lambda x: x['timestamp'], reverse=True)
        fees_2_weeks_ago = [x for x in pools_shifted if x['pool']['id'] == pool]
        fees_2_weeks_ago.sort(key=lambda x: x['timestamp'], reverse=True)
        # Take first element of list, which is the most recent snapshot
        if not current_fees_snapshots or not fees_2_weeks_ago:
            continue
        pool_snapshot_now = current_fees_snapshots[0]
        pool_snapshot_2_weeks_ago = fees_2_weeks_ago[0]
        # Calculate fees
        pool_fee = float(pool_snapshot_now['protocolFee']) - float(pool_snapshot_2_weeks_ago['protocolFee'])
        pool_swap_fee = float(pool_snapshot_now['swapFees']) - float(pool_snapshot_2_weeks_ago['swapFees'])
        # Now we need to collect token fee info. Let's start with BPT tokens, which is Balancer pool token. Notice,
        # That totalProtocolFeePaidInBPT can be null, so we need to check for that
        bpt_token_fee = 0
        bpt_price_usd = arb_bpt_twap_prices[pool] if chain == 'arbitrum' else mainnet_bpt_twap_prices[
            pool] if chain == 'mainnet' else polygon_bpt_twap_prices[pool] if chain == 'polygon' else \
            base_bpt_twap_prices[pool] if chain == 'base' else gnosis_bpt_twap_prices[pool] if chain == 'gnosis' else \
            avax_bpt_twap_prices[pool]
        if bpt_price_usd is None:
            bpt_price_usd = 0
        if pool_snapshot_now['pool']['totalProtocolFeePaidInBPT'] is not None and pool_snapshot_2_weeks_ago['pool'][
            'totalProtocolFeePaidInBPT'] is not None:
            bpt_token_fee = float(pool_snapshot_now['pool']['totalProtocolFeePaidInBPT']) - float(
                pool_snapshot_2_weeks_ago['pool']['totalProtocolFeePaidInBPT'])
            token_fees[pool_snapshot_now['pool']['symbol']].append({
                'token': pool_snapshot_now['pool']['symbol'],
                'token_fee': bpt_token_fee,
                'token_price': bpt_price_usd,
                'token_fee_in_usd': Decimal(bpt_token_fee) * bpt_price_usd,
                'token_addr': pool_snapshot_now['pool']['address'],
                'time_from': arb_datetime_2_weeks_ago,
                'time_to': arb_datetime_now,
                'chain': chain,
            })
        # Now collect fee info about fees paid in pool tokens. Pool tokens fee info is in pool.tokens dictionary. This will be separate dictionary
        else:
            bpt_price_usd = 0
            for token_data in pool_snapshot_now['pool']['tokens']:
                token_data_2_weeks_ago = \
                    [t for t in pool_snapshot_2_weeks_ago['pool']['tokens'] if t['address'] == token_data['address']][0]
                token_fee = float(token_data.get('paidProtocolFees', None)) - float(
                    token_data_2_weeks_ago.get('paidProtocolFees', None))
                # Get twap token price from CoinGecko
                token_price = fetch_token_price_balgql(token_data['address'], chain, start_date) or 0
                token_fees[pool_snapshot_now['pool']['symbol']].append({
                    'token': token_data['symbol'],
                    'token_fee': token_fee,
                    'token_price': token_price,
                    'token_fee_in_usd': Decimal(token_fee) * token_price if token_price is not None else 0,
                    'token_addr': token_data['address'],
                    'time_from': arb_datetime_2_weeks_ago,
                    'time_to': arb_datetime_now,
                    'chain': chain,
                })
        # Calculate non-BPT fees in USD
        fees[pool_snapshot_now['pool']['symbol']] = {
            'pool_fee': round(pool_fee, 2),
            'swap_fee': round(pool_swap_fee, 2),
            'bpt_token_fee': round(bpt_token_fee, 2),
            # Get fee in USD by multiplying bpt_token_fee by price of BPT token taken from twap_bpt_price
            'bpt_token_fee_in_usd': round(Decimal(bpt_token_fee) * bpt_price_usd, 2),
            'token_fees_in_usd': round(sum([x['token_fee_in_usd'] for x in
                                            token_fees[
                                                pool_snapshot_now['pool']['symbol']]]) if bpt_price_usd == 0 else 0, 2),
            'time_from': arb_datetime_2_weeks_ago,
            'time_to': arb_datetime_now,
            'chain': chain,
            'token_fees': token_fees[pool_snapshot_now['pool']['symbol']],
        }
    return fees


arb_fees = collect_fee_info(ARB_CORE_POOLS, 'arbitrum', arbi_pool_snapshots_now,
                            arbi_pool_snapshots_2_weeks_ago, arb_datetime_now)
mainnet_fees = collect_fee_info(MAINNET_CORE_POOLS, 'mainnet', mainnet_pool_snapshots_now,
                                mainnet_pool_snapshots_2_weeks_ago, mainnet_datetime_now)
polygon_fees = collect_fee_info(POLYGON_CORE_POOLS, 'polygon', polygon_pool_snapshots_now,
                                polygon_pool_snapshots_2_weeks_ago, poly_datetime_now)
base_fees = collect_fee_info(BASE_CORE_POOLS, 'base', base_pool_snapshots_now, base_pool_snapshots_2_weeks_ago,
                             base_datetime_now)
gnosis_fees = collect_fee_info(GNOSIS_CORE_POOLS, 'gnosis', gnosis_pool_snapshots_now,
                               gnosis_pool_snapshots_2_weeks_ago, gnosis_datetime_now)
avax_fees = collect_fee_info(AVAX_CORE_POOLS, 'avalanche', avax_pool_snapshots_now,
                             avax_pool_snapshots_2_weeks_ago, avax_datetime_now)
# Convert to dataframe, sort by chain and pool fee
joint_fees = {**arb_fees, **mainnet_fees, **polygon_fees, **base_fees, **gnosis_fees, **avax_fees}
joint_fees_df = pd.DataFrame.from_dict(joint_fees, orient='index')

In [11]:
# Remove `token_fees` field from dataframe, as it's too big
joint_fees_df_copy = joint_fees_df.drop(columns=['token_fees'])
# Display all rows in dataframe
pd.set_option('display.max_rows', 1000)
joint_fees_df_copy.sort_values(by=['chain', 'pool_fee'], ascending=False)

,pool_fee,swap_fee,bpt_token_fee,bpt_token_fee_in_usd,token_fees_in_usd,time_from,time_to,chain
maticX-WMATIC-BPT,1789.92,327.36,3377.40,1808.37,0,2023-09-01 21:00:00,2023-09-15 21:00:00,polygon
ECLP-WMATIC-stMATIC,1495.34,248.25,2781.91,1501.92,0,2023-09-01 21:00:00,2023-09-15 21:00:00,polygon
ECLP-WMATIC-MATICX,403.27,71.43,755.16,406.09,0,2023-09-01 21:00:00,2023-09-15 21:00:00,polygon
stMATIC-WMATIC-BPT,2.58,0.92,4.62,2.47,0,2023-09-01 21:00:00,2023-09-15 21:00:00,polygon
wstETH-WETH-BPT,0.00,0.00,0.00,0.00,0.00,2023-09-01 21:00:00,2023-09-15 21:00:00,polygon
B-rETH-STABLE,15892.61,2207.43,0.00,0.00,15796.29,2023-09-01 21:00:00,2023-09-15 21:00:00,mainnet
50rETH-50BADGER,9939.15,3065.68,166.20,9828.69,0,2023-09-01 21:00:00,2023-09-15 21:00:00,mainnet
swETH-WETH-BPT,5505.42,640.81,3.43,5513.21,0,2023-09-01 21:00:00,2023-09-15 21:00:00,mainnet
ankrETH/wstETH,3867.44,126.65,2.32,3789.10,0,2023-09-01 21:00:00,2023-09-15 21:00:00,mainnet
20WETH-80ALCX,2393.07,5322.24,83.27,2075.09,0,2023-09-01 21:00:00,2023-09-15 21:00:00,mainnet


## Now let's calculate bribes paid to the pools

In [12]:
from notebooks import calculate_aura_vebal_share

aura_vebal_share = calculate_aura_vebal_share(eth_web3, mainnet_block)

# Bribes are split per chain and per pool, with each pool getting a share of the bribe proportional to its share of fees
# paid by all pools on that chain. For example, if pool A paid 10% of all fees on Arbitrum, it will get 10% of the bribes. That 10% will be distributed between aura and vebal, proportional to their share of the bribe.
FEE = Decimal(0.5)  # 50% goes to fees


# Let's calculate share of fees paid by each pool on each chain
def calc_and_split_bribes(fees: Dict, chain: str) -> Dict[str, Dict]:
    pool_bribs = {}
    for pool, data in fees.items():
        pool_fees = data['bpt_token_fee_in_usd'] + data['token_fees_in_usd']
        # Split fees between aura and bal fees
        pool_bribs[pool] = {
            "chain": chain,
            "aura_bribes": pool_fees * FEE * aura_vebal_share,
            "bal_bribes": pool_fees * FEE * (1 - aura_vebal_share),
            "pool_total": pool_fees,
            "fees_taken": pool_fees * FEE,
        }
    return pool_bribs


mainnet_bribes = calc_and_split_bribes(mainnet_fees, 'mainnet')
arb_bribes = calc_and_split_bribes(arb_fees, 'arbitrum')
polygon_bribes = calc_and_split_bribes(polygon_fees, 'polygon')
base_bribes = calc_and_split_bribes(base_fees, 'base')
gnosis_bribes = calc_and_split_bribes(gnosis_fees, 'gnosis')
avax_bribes = calc_and_split_bribes(avax_fees, 'avalanche')
# Convert to dataframe
joint_bribes_data = {**arb_bribes, **mainnet_bribes, **polygon_bribes, **base_bribes, **gnosis_bribes, **avax_bribes}
# Sort by chain:
joint_bribes_data = {k: v for k, v in sorted(joint_bribes_data.items(), key=lambda item: item[1]['chain'])}
joint_bribes_df = pd.DataFrame.from_dict(joint_bribes_data, orient='index')
# Sort by chain
# Dump into csv and prefix with dates
joint_bribes_df.to_csv(f'../data/bribs/bribes_{arb_datetime_2_weeks_ago.date()}_{arb_datetime_now.date()}.csv')
joint_bribes_df

,chain,aura_bribes,bal_bribes,pool_total,fees_taken
wstETH/rETH/cbETH,arbitrum,407.6311676804805896312019791,617.0538323195194103687980209,2049.37,1024.685
4POOL-BPT,arbitrum,94.63729500807363225802543691,143.2577049919263677419745631,475.79,237.895
RDNT-WETH,arbitrum,2252.924159058506490539420572,3410.375840941493509460579428,11326.60,5663.300
wstETH/rETH/sfrxETH,arbitrum,0.003978111982516388837849699948,0.006021888017483611162150300052,0.02,0.010
sAVAX-WAVAX-BPT,avalanche,480.0347948182701246744854430,726.6552051817298753255145570,2413.38,1206.690
ggAVAX-WAVAX-BPT,avalanche,83.25392757010298559851852051,126.0260724298970144014814795,418.56,209.280
yyAVAX-WAVAX-BPT,avalanche,157.3581975804182768699827311,238.2018024195817231300172689,791.12,395.560
USDC-USDT-BPT,avalanche,24.45942152450201676951888013,37.02557847549798323048111987,122.97,61.485
BPT,avalanche,0E-31,0E-31,0.00,0.000
rETH-WETH-BPT,base,0E-31,0E-31,0.00,0.000
